In [ ]:
#DON'T RENAME ME - I'M CHANGED EXTERNALLY
OUTPUT="output/"
K0 =
G0 =
G1 =
B0 =
KX =
Xref =
Xfreq =
Dref =
Dfreq = 


In [ ]:
from IPython.display import HTML
HTML('code_toggle.html')

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from pyrap.tables import table as tbl
import matplotlib.dates as mdates
from casacore.quanta import quantity
import casacore.quanta as qa
from casacore.measures import measures
import datetime
import os
from ipywidgets import HTML
from IPython.core.display import display

In [ ]:
def convertMJD2unix(time):
    res = []
    for ti, t in enumerate(time):
        tqnt = quantity(str(float(t))+'s')
        res.append(datetime.datetime.utcfromtimestamp(tqnt.to_unix_time()))
    return res

# Delays

In [ ]:
if os.path.exists(K0):
    with tbl(K0) as t:
        delays = t.getcol("FPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")
    with tbl("%s::ANTENNA" % K0) as t:
        antnames = t.getcol("NAME")
    delays[flags] = np.nan


    hrloc = mdates.HourLocator()
    minloc = mdates.MinuteLocator()  
    dtFmt = mdates.DateFormatter('%hh%mm%ss')

    for corr in xrange(delays.shape[2]):    
        collections = []
        collections_time = []
        pcmax = -np.inf
        pcmin = np.inf
        for a in np.unique(ants):
            asel = ants == a
            unflagged = np.logical_not(flags[:, 0, corr][asel])
            collections.append(delays[:, 0, corr][asel][unflagged])
            pcmax = max(pcmax, np.nanpercentile(delays[:, 0, corr][asel][unflagged],98))
            pcmin = min(pcmin, np.nanpercentile(delays[:, 0, corr][asel][unflagged],2))
            collections_time.append(time[asel][unflagged])
        labels=[antnames[ai] for ai in np.unique(ants)]
        plt.figure(figsize=(25, 6))
        plt.title("Correlation %d" % corr)
        plt.boxplot(collections, 0, '', labels=labels)
        plt.ylabel("Delay (ns)")
        plt.show()
        fig, ax = plt.subplots(figsize=(25, 6))
        for t,a,aname in zip(collections_time, collections, labels):
            ax.plot(convertMJD2unix(t), a, label=aname)
        ax.set_ylabel("Delay (ns) [98%]")
        ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
        ax.legend(loc = (1.01,0))
        ax.grid(True)
        hfmt = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(hfmt)
        limmean = np.nanmean(delays)
        lim = np.nanstd(delays)
        ax.set_ylim(pcmin, pcmax)
        plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Bandpass gain stability

In [ ]:
if os.path.exists(G0):
    with tbl(G0) as t:
        bpgain = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")
    with tbl("%s::ANTENNA" % G0) as t:
        antnames = t.getcol("NAME")
    bpgain[flags] = np.nan
    for corr in xrange(bpgain.shape[2]):
        collections = []
        collections_time = []
        for a in np.unique(ants):
            asel = ants == a
            unflagged = np.logical_not(flags[:, 0, corr][asel])
            collections.append(bpgain[:, 0, corr][asel][unflagged])
            collections_time.append(time[asel][unflagged])
        labels=[antnames[ai] for ai in np.unique(ants)]
        fig, ax = plt.subplots(figsize=(25, 6))
        for t,a,aname in zip(collections_time, collections, labels):
            ax.plot(convertMJD2unix(t), 10*np.log10(np.abs(a)), label=aname)
        ax.set_title("Correlation %d" % corr)
        ax.set_ylabel("Amplitude [dB]")
        ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
        ax.legend(loc = (1.01,0))
        ax.grid(True)
        hfmt = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(hfmt)
        plt.show()

        fig, ax = plt.subplots(figsize=(25, 6))
        for t,a,aname in zip(collections_time, collections, labels):
            ax.plot(convertMJD2unix(t), np.rad2deg(np.angle(a)), label=aname)
        ax.set_title("Correlation %d" % corr)
        ax.set_ylabel("Phase [deg]")
        ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
        ax.legend(loc = (1.01,0))
        ax.grid(True)
        hfmt = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(hfmt)
        plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Gain phase stability (all fields)

In [ ]:
if os.path.exists(G1):
    with tbl(G1) as t:
        bpgain = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")

    with tbl("%s::ANTENNA" % G1) as t:
        antnames = t.getcol("NAME")

    bpgain[flags] = np.nan
    for corr in xrange(bpgain.shape[2]):
        collections = []
        collections_time = []
        for a in np.unique(ants):
            asel = ants == a
            unflagged = np.logical_not(flags[:, 0, corr][asel])
            collections.append(bpgain[:, 0, corr][asel][unflagged])
            collections_time.append(time[asel][unflagged])
        labels=[antnames[ai] for ai in np.unique(ants)]

        fig, ax = plt.subplots(figsize=(25, 6))
        for t,a,aname in zip(collections_time, collections, labels):
            ax.plot(convertMJD2unix(t), np.rad2deg(np.angle(a)), label=aname)
        ax.set_title("Correlation %d" % corr)
        ax.set_ylabel("Phase [deg]")
        ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
        ax.legend(loc = (1.01,0))
        ax.grid(True)
        hfmt = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(hfmt)
        plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Bandpass

In [ ]:
if os.path.exists(B0):
    with tbl(B0) as t:
        bp = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")

    with tbl("%s::ANTENNA" % B0) as t:
        antnames = t.getcol("NAME")
    with tbl("%s::SPECTRAL_WINDOW" % B0) as t:
        freqs = t.getcol("CHAN_FREQ")/1.0e6
    bp[flags] = np.nan
    for corr in xrange(bp.shape[2]):
        collections = []
        for a in np.unique(ants):
            asel = ants == a
            collections.append(bp[:, :, corr][asel])
        labels=[antnames[ai] for ai in np.unique(ants)]

        fig, ax = plt.subplots(figsize=(25, 6))

        for a,aname in zip(collections, labels):
            ax.scatter(np.tile(freqs, (1, a.shape[0])), 
                    10*np.log10(np.abs(a)),
                    s=1.5,
                    label=aname)

        ax.set_title("Correlation %d" % corr)
        ax.set_ylabel("Amplitude [dB]")
        ax.set_xlabel("Frequency (MHz)")
        ax.legend(loc = (1.01,0))
        ax.grid(True)

        fig, ax = plt.subplots(figsize=(25, 6))
        for a,aname in zip(collections, labels):
            ax.scatter(np.tile(freqs, (1, a.shape[0])), 
                    np.rad2deg(np.angle(a)), 
                    s=1.5,
                    label=aname)
        ax.set_title("Correlation %d" % corr)
        ax.set_ylabel("Phase [deg]")
        ax.set_xlabel("Frequency (MHz)")
        ax.legend(loc = (1.01,0))
        ax.grid(True)

        plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Crosshand delays

In [ ]:
if os.path.exists(KX):
    with tbl(KX) as t:
        delays = t.getcol("FPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")
    with tbl("%s::ANTENNA" % KX) as t:
        antnames = t.getcol("NAME")
    delays[flags] = np.nan


    hrloc = mdates.HourLocator()
    minloc = mdates.MinuteLocator()  
    dtFmt = mdates.DateFormatter('%hh%mm%ss')


    collections = []
    collections_time = []
    pcmax = -np.inf
    pcmin = np.inf
    for a in np.unique(ants):
        asel = ants == a
        unflagged = np.logical_not(flags[:, 0, 0][asel])
        collections.append(delays[:, 0, 0][asel][unflagged])
        pcmax = max(pcmax, np.nanpercentile(delays[:, 0, 0][asel][unflagged],98))
        pcmin = min(pcmin, np.nanpercentile(delays[:, 0, 0][asel][unflagged],2))
        collections_time.append(time[asel][unflagged])
    labels=[antnames[ai] for ai in np.unique(ants)]
    plt.figure(figsize=(25, 6))
    plt.title("Crosshand delays")
    plt.boxplot(collections, 0, '', labels=labels)
    plt.ylabel("Delay (ns)")
    plt.show()
    fig, ax = plt.subplots(figsize=(25, 6))
    for t,a,aname in zip(collections_time, collections, labels):
        ax.plot(convertMJD2unix(t), a, label=aname)
    ax.set_ylabel("Delay (ns) [98%]")
    ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
    ax.legend(loc = (1.01,0))
    ax.grid(True)
    hfmt = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(hfmt)
    limmean = np.nanmean(delays)
    lim = np.nanstd(delays)
    ax.set_ylim(pcmin, pcmax)
    plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Crosshand phase gain stability

In [ ]:
if os.path.exists(Xref):
    with tbl(Xref) as t:
        bpgain = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")
    with tbl("%s::ANTENNA" % Xref) as t:
        antnames = t.getcol("NAME")
    bpgain[flags] = np.nan
    for corr in xrange(bpgain.shape[2]):
        collections = []
        collections_std = []
        collections_time = []
        for a in np.unique(ants):
            asel = ants == a
            bpgain[flags] = np.nan
            ang = np.angle(bpgain[asel, :, corr])
            collections.append(np.nanmedian(ang, axis=1))
            collections_std.append((np.nanpercentile(ang, 75.0, axis=1) - np.nanpercentile(ang, 25.0, axis=1))*0.5)
            collections_time.append(time[asel])
        labels=[antnames[ai] for ai in np.unique(ants)]

        fig, ax = plt.subplots(figsize=(25, 6))
        for t,a,s,aname in zip(collections_time, collections, collections_std, labels):
            ax.errorbar(convertMJD2unix(t), np.rad2deg(a), yerr=np.rad2deg(s), label=aname)
        ax.set_title("Crosshand phase DC")
        ax.set_ylabel("Phase [deg]")
        ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
        ax.legend(loc = (1.01,0))
        ax.grid(True)
        hfmt = mdates.DateFormatter('%H:%M')
        ax.xaxis.set_major_formatter(hfmt)
        plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Crosshand phase

In [ ]:
if os.path.exists(Xfreq):
    with tbl(Xfreq) as t:
        xfsols = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")

    with tbl("%s::ANTENNA" % Xfreq) as t:
        antnames = t.getcol("NAME")
    with tbl("%s::SPECTRAL_WINDOW" % Xfreq) as t:
        freqs = t.getcol("CHAN_FREQ")/1.0e6
    xfsols[flags] = np.nan
    collections = []
    for a in np.unique(ants):
        asel = ants == a
        collections.append(xfsols[:, :, corr][asel])

    labels=[antnames[ai] for ai in np.unique(ants)]

    fig, ax = plt.subplots(figsize=(25, 6))
    for a,aname in zip(collections, labels):
        ax.scatter(np.tile(freqs, (1, a.shape[0])), 
                np.rad2deg(np.angle(a)), 
                s=1.5,
                label=aname)
    ax.set_title("Crosshand phase")
    ax.set_ylabel("Phase [deg]")
    ax.set_xlabel("Frequency (MHz)")
    ax.legend(loc = (1.01,0))
    ax.grid(True)

    plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# First order leakage gain stability

In [ ]:
if os.path.exists(Dref):
    with tbl(Dref) as t:
        dgain = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")
    with tbl("%s::ANTENNA" % Dref) as t:
        antnames = t.getcol("NAME")
    dgain[flags] = np.nan

    collections = []
    collections_time = []
    for a in np.unique(ants):
        asel = ants == a
        unflagged = np.logical_not(flags[:, 0, 0][asel])
        collections.append(dgain[:, 0, 0][asel][unflagged])
        collections_time.append(time[asel][unflagged])
    labels=[antnames[ai] for ai in np.unique(ants)]

    fig, ax = plt.subplots(figsize=(25, 6))
    for t,a,aname in zip(collections_time, collections, labels):
        ax.plot(convertMJD2unix(t), 10*np.log10(np.abs(a)), label=aname)
    ax.set_title("Leakage gain")
    ax.set_ylabel("Amplitude [dB]")
    ax.set_xlabel("Time (start: %s)" % str(convertMJD2unix([np.min(time)])[0]))
    ax.legend(loc = (1.01,0))
    ax.grid(True)
    hfmt = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(hfmt)
    plt.show()

    plt.figure(figsize=(25, 6))
    plt.title("DC leakage")
    plt.boxplot([10*np.log10(np.abs(c)) for c in collections], 0, '', labels=labels)
    plt.ylabel("DC leakage")
    plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))

# Leakage

In [ ]:
if os.path.exists(Dfreq):
    with tbl(Dfreq) as t:
        dfsols = t.getcol("CPARAM")
        ants = t.getcol("ANTENNA1")
        field = t.getcol("FIELD_ID")
        flags = t.getcol("FLAG")
        time = t.getcol("TIME")

    with tbl("%s::ANTENNA" % Dfreq) as t:
        antnames = t.getcol("NAME")
    with tbl("%s::SPECTRAL_WINDOW" % Dfreq) as t:
        freqs = t.getcol("CHAN_FREQ")/1.0e6
    dfsols[flags] = np.nan
    collections = []
    for a in np.unique(ants):
        asel = ants == a
        collections.append(dfsols[:, :, corr][asel])

    labels=[antnames[ai] for ai in np.unique(ants)]

    fig, ax = plt.subplots(figsize=(25, 6))
    for a,aname in zip(collections, labels):
        ax.scatter(np.tile(freqs, (1, a.shape[0])), 
                10*np.log10(np.abs(a)), 
                s=1.5,
                label=aname)
    ax.set_title("Leakage")
    ax.set_ylabel("Leakage [dB]")
    ax.set_xlabel("Frequency (MHz)")
    ax.legend(loc = (1.01,0))
    ax.grid(True)

    plt.show()
else: display(HTML("<p color='#ff0000'> Table does not exist. Step was probably not run. </p>"))